In [2]:
import numpy as np
import numpy as np
from cadCAD.configuration import Experiment
from cadCAD.configuration.utils import config_sim
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
import pandas as pd


# State Variables
state = {
    'volume': 1000,
    'liquidity': 12000,
    'staking_score': 100,
    'holding': 1200,
    'kpi_score': 0,
    'score_index': 0,
    'category': 'Regular'
}

# Model Parameters
params = {
    'volume_weight': [10],
    'liquidity_weight': [20],
    'staking_weight': [30],
    'holding_weight': [40]
}

category_thresholds = {
    'platinum_threshold': 0.75,
    'gold_threshold': 0.5,
    'silver_threshold': 0.25
}

# Policy Functions
def calculate_kpi_score(params, step, history, current_state):
    kpi_score = params['volume_weight'] * current_state['volume'] + \
                params['liquidity_weight'] * current_state['liquidity'] + \
                params['staking_weight'] * current_state['staking_score'] + \
                params['holding_weight'] * current_state['holding']
    return {'kpi_score': kpi_score}

def calculate_user_score_index(params, step, history, current_state):
    total_kpi_scores = sum(current_state['kpi_score'].values())
    score_index = current_state['kpi_score'] / total_kpi_scores
    return 'score_index',score_index

def categorize_users(user_score_index):
    if user_score_index > category_thresholds['platinum_threshold']:
        category = 'Platinum'
    elif user_score_index > category_thresholds['gold_threshold']:
        category = 'Gold'
    elif user_score_index > category_thresholds['silver_threshold']:
        category = 'Silver'
    else:
        category = 'Regular'
    return {'category':category}

# State Update Functions
def update_kpi_score(user,params, step, history, current_state, input_):
    current_state['kpi_score'] = input_['kpi_score']
    return ('kpi_score', {**state['kpi_score'], user: current_state['kpi_score']})

def update_user_score_index(user,params, step, history, current_state, input_):
   
    current_state['score_index'] = input_['score_index']
    return ('user_score_index', {**state['user_score_index'], user: current_state['score_index']})

def update_category(user,params, step, history, current_state, input_):
    current_state['category'] = input_['category']
    return ('category', {**state['category'], user: current_state['category']})

# Partial State Update Blocks
partial_state_update_blocks = [
    {
        'policies': {
            'calculate_kpi_score': calculate_kpi_score
        },
        'variables': {
            'kpi_score': update_kpi_score
        }
    },
    {
        'policies': {
            'calculate_user_score_index': calculate_user_score_index
        },
        'variables': {
            'score_index': update_user_score_index
        }
    },
    {
        'policies': {
            'categorize_users': categorize_users
        },
        'variables': {
            'category': update_category
        }
    }
]

# Simulation Parameters
sim_config = {
    'N': 1,  # number of Monte Carlo runs
    'T': range(1000),  # number of timesteps
    'M': params  # model parameters
}

sim_params = config_sim(sim_config)

# Configuring and running the simulation
exp = Experiment()
exp.append_configs(
    sim_configs=sim_params,
    initial_state=state,
    partial_state_update_blocks=partial_state_update_blocks
)

exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(context=exec_mode.local_mode)

simulation = Executor(exec_context=local_mode_ctx, configs=exp.configs)
raw_system_events, tensor_field, sessions = simulation.execute()
df = pd.DataFrame(raw_system_events)



                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 1000, 4, 1, 7)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (1000, 4, 1, 7)
Execution Method: local_simulations
Execution Mode: single_threaded


TypeError: 'NoneType' object is not subscriptable